# 네이버 블로그 크롤링 (네이버 API 이용)

## 들어가며...



## 1) 크롤링? 파싱?

### 1-1) 크롤링

- '웹 크롤러'라는 단어에서 유래되었음.
- 크롤러란 조직적, 자동화된 방법으로 WWW를 탐색하는 컴퓨터 프로그램.
- 크롤링은 크롤러가 하는 작업을 부르는 말.
- 여러 인터넷 사이트의 페이지(문서,html 등)를 수집해서 분류하는 것.
- 대체로 찾아낸 데이터를 저장한 후 쉽게 찾을 수 있게 인덱싱 수행.

### 1-2) 파싱

- 파싱이란 어떤 페이지(문서, html 등)에서 내가 원하는 데이터를 특정 패턴이나 순서로 추출하여 정보를 가공하는 것.
- 파싱이란 일련의 문자열을 의미있는 '토큰'으로 분해하고 이들로 이루어진 '파스 트리'를 만드는 과정.
- 입력 토큰에 내제된 자료 구조를 빌드하고 문법을 검사하는 역할을 함.

## 2) 애플리케이션 등록 ( API 이용신청)

![애플리케이션 등록_1](https://user-images.githubusercontent.com/51112316/61574953-2f655d80-ab01-11e9-8b8f-cc74183302da.jpg)
![애플리케이션 등록_2](https://user-images.githubusercontent.com/51112316/61574954-31c7b780-ab01-11e9-9274-e1d237547d94.jpg)

### 2-1) 네이버API  예제 사용

- 네이버개발자 -> Products -> 서비스API -> 검색 -> 개발 가이드 보기 -> 0.API호출예제 -> Python

#### [API호출예제](https://developers.naver.com/docs/search/blog/)

~~~python
import java.io.BufferedReader;
import java.io.InputStreamReader;
import java.net.HttpURLConnection;
import java.net.URL;
import java.net.URLEncoder;

public class APIExamSearchBlog {

    public static void main(String[] args) {
        String clientId = "YOUR_CLIENT_ID";//애플리케이션 클라이언트 아이디값";
        String clientSecret = "YOUR_CLIENT_SECRET";//애플리케이션 클라이언트 시크릿값";
        try {
            String text = URLEncoder.encode("그린팩토리", "UTF-8");
            String apiURL = "https://openapi.naver.com/v1/search/blog?query="+ text; // json 결과
            //String apiURL = "https://openapi.naver.com/v1/search/blog.xml?query="+ text; // xml 결과
            URL url = new URL(apiURL);
            HttpURLConnection con = (HttpURLConnection)url.openConnection();
            con.setRequestMethod("GET");
            con.setRequestProperty("X-Naver-Client-Id", clientId);
            con.setRequestProperty("X-Naver-Client-Secret", clientSecret);
            int responseCode = con.getResponseCode();
            BufferedReader br;
            if(responseCode==200) { // 정상 호출
                br = new BufferedReader(new InputStreamReader(con.getInputStream()));
            } else {  // 에러 발생
                br = new BufferedReader(new InputStreamReader(con.getErrorStream()));
            }
            String inputLine;
            StringBuffer response = new StringBuffer();
            while ((inputLine = br.readLine()) != null) {
                response.append(inputLine);
            }
            br.close();
            System.out.println(response.toString());
        } catch (Exception e) {
            System.out.println(e);
        }
    }
}
~~~

#### 문제 발생 :  

#### 네이버 블로그 형식 맞춰 긁어왔음.
                  
#### 사실은 요약된 내용.

#### 모든 내용을 다 보고싶으면 개발이 필요.

# 네이버 블로그 크롤링

In [18]:
import re
import json
import math
import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup

naver_Client_id = "E4zDnlSmVAnJ7dvIOUUx"
naver_Client_secret = "lLxEw7DWbB"

In [24]:
def get_blog_count(query,display) :
    encode_query=urllib.parse.quote(query)
    search_url="https://openapi.naver.com/v1/search/blog?query="+ encode_query
    # 위의 https://~ 는 따온 API호출예제에 있음.
    request=urllib.request.Request(search_url)
    # 이렇게 하면 내가 만들었던 url 형태로 요청한다.
    # 그러나 이렇게 하면 API로부터 거부당한다.
    # 왜냐하면 ID, Secret을 주지 않았기 때문에 아래와 같이 이것을 입력해준다.
    
    request.add_header("X-Naver-Client-Id",naver_Client_id)
    request.add_header("X-Naver-Client-Secret",naver_Client_secret)
    
    response=urllib.request.urlopen(request)
    response_code=response.getcode()
    
    # code를 받아와서 웹에서 정상적으로 접속했을 때 200번을 준다.
    if response_code is 200 :
        response_body = response.read()  # 실제로 읽어 들인다.
        response_body_dict = json.loads(response_body.decode('utf-8'))
        # 네이버에서 넘겨줄 때 json으로 넘겨주는데 이것을 전부 '딕셔너리'로 저장한다.
        
        print('Last build date :' + str(response_body_dict['lastBuildDate']))
        # json으로 넘겨받을 것들 중에서 최종날짜를 출력
        print('Total :' + str(response_body_dict['total']))  # 관측개수
        print('Start :' + str(response_body_dict['start']))
        print('Display :' + str(response_body_dict['display']))
        #이렇게 넘어온 것을 다 출력해본다.
        
        # 검색된 결과가 0 일 경우        
        if response_body_dict['total']== 0:
            blog_count = 0
        else :
            blog_total=math.ceil(response_body_dict['total']/int(display))
            # 위에서 한 번에 10개씩 출력한다고 했으니 나눠주어서 개수를 맞춰준다.
        
            if blog_total >= 1000 :
                blog_count = 1000
                # 최대 1000개로 제한이 있으니 1000개를 넘었을 때는 1000개로 지정한다.
            else : 
                blog_count = blog_total
        
            print('Blog total :' + str(blog_total))
            print('Blog count : ' + str(blog_count))
        
        return blog_count

- 블로그의 모든 내용을 가져오는 함수[(API사용예제)는 샘플 내용만 가져온다.]



In [39]:
def get_blog_post(query,display,start_index,sort):
    global no, fs
    #get_blog_count(query,display)를 정의할 때 사용했던 것과 동일하게 사용(아래 7문장)
    encode_query = urllib.parse.quote(query)
    # 동일하게 사용하나 search_url에는 옵션을 조금 더 추가해 준다.
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query +"&display=" + str(display) + "&start=" + str(start_index) + "sort=" + sort
    request=urllib.request.Request(search_url)
    
    request.add_header("X-Naver-Client-Id",naver_Client_id)
    request.add_header("X-Naver-Client-Secret",naver_Client_secret)
    
    response=urllib.request.urlopen(request)
    response_code=response.getcode()
    
    #print(response_code) 이분은 윗부분까지 해서 접속이 잘되었는지를 확인하는 것.
        
    if response_code is 200:
        response_body=response.read()
        response_body_dict=json.loads(response_body.decode('utf-8'))
        # json이 해당하는 블로그가 여러개 나오면 전부 다 하나의 큰 덩어리로 주는데
        # 나눠서 하나씩 접근 하기 위해 아래의 코드를 짜줌.
        for item_index in range(0,len(response_body_dict['items'])):
            # 혹시 태그부분이 따라올 수 있기 때문에
            # 그것을 지우고 사용하기 위해  re(regular expression)으로 제거를 해준다.
            # 즉 태그를 없애주는 정규표현식을 이용한다.
            try :
                remove_html_tag = re.compile('<.*?>')
                title = re.sub(remove_html_tag,'',response_body_dict['items'][item_index]['title'])
                #또한 주소가 따라오는데 이건 필요 없는 것이다.그러므로 없애준다.
                link = response_body_dict['items'][item_index]['link'].replace("amp;","")
                # description에도 마찬가지로 태그가 따라올 수 있기 때문에 제거해준다.
                description = re.sub(remove_html_tag,'',response_body_dict['items'][item_index]['description'])
                blogger_name = response_body_dict['items'][item_index]['bloggername']
                #  포스트(개시글)의 주소가 아니라 블로그 자체의 링크를 가져옴.
                blogger_link = response_body_dict['items'][item_index]['bloggerlink']
                post_date = response_body_dict['items'][item_index]['postdate']
                #아래처럼 변환해서 가져와도 되고 그냥 주는대로 가져와도 된다.
                #post_date = datetime.datetime.strptime(response_body_dict[['item'][item_index]['postdate'],"%Y%m%d").strtime("%Y.%m.%d")
                
               # 이제 json에서 오는 결과를 하나씩 items에 다 들어가 있다. 그것을 이제 다 끄집어서 가져온다.
                no +=1
                print("-------------------------------------------------") #각각의 게시글을 구분시킴
                print("#"+ str(no))  #몇번째 게시글인지 출력
                print("Title :"+ title)
                print("Link :"+ link)
                print("Description :"+ description)
                print("Blogger Name :"+ blogger_name)
                print("Blogger Link :"+ blogger_link)
                print("Post Date :"+ post_date)
                
                                                                              
                post_code = request.get(link)
                #  저 정보들은 사실 네이버 오픈 API에서 제공해주는 정보다.
                # 그런데 우리가 필요한 것은 실제 블로그에 포스트 된 내용이 필요하기 때문에
                # 뷰티풀숲을 이용해 가져온다. 
                post_text = post_code.text
                post_soup = BeautifulSoup(post_text,'lxml')
                
                # 네이버 블로그의 문제점 : 마우스를 긁어오지 못하게 iframe으로 만들어 놓은게 있다.
                #그래서 iframe#mainFrame 이라는 html 부분을 들어가서
                # 그 부분의 실제 post url을 뽑을 수 있다.
                # 그냥 가져와선 안되고 그 url에서 소스부분(src)만 가져온다.
                # 그래서 그걸로 접속해서 실제적인 블로그 포스트에 대한 내용을 긁어오기 때문에
                # 여기에 다시 한번 "blog_post_url"이라는 주소로 접근을 한다.
                # 그리고 그것들에 대한 정보를 가져온다.
                for mainFrame in post_soup.select('iframe#mainFrame'):
                    blog_post_url = "http://blog.naver.com"+mainFrame.get('src')
                    blog_post_code = requests.get(blog_post_url)
                    blog_post_text = blog_post_code.text
                    blog_post_soup = BeautifulSoup(blog_post_text,'lxml')
                    
                    # 블로그 전체의 여러가지 메뉴들도 있고 그런 것 제외하고 postViewArea라는 부분에
                    # 실제 포스트 내용이 들어있다. 그래서 여기로 선택한다. 이게 진짜 블로그 포스트의 컨텐트다. 
                    for blog_post_content in blog_post_soup.select("div#postViewArea"):
                        blog_post_content_text = blog_post_content.get_text()
                        blog_post_full_contents = str(blog_post_content_text) #컨텐트를 string으로 캐스팅해서 가져온다.
                        blog_post_full_contents = blog_post_full_contents.replace("\n\n",'\n') 
                        #요약된 description 만 주는게 아니라 이젠 진짜로 된 포스트를 준다.
                        #print("blog_post_contents : " + blog_post_full_contents+"\n")
                        #전체 내용을 print 하면 너무 많기 때문에 위의 코드로 하지않고 파일로 저장한다.
                        #이를 통해 이후에 데이터 분석 등에 사용한다.
                        fs.write(blog_post_full_contents+'\n')
                        fs.write("-----------------------------------\n")
            #만약에 못할 경우 그냥 넘어가는 코드.
            except: 
                item_index+=1

In [40]:
if __name__=='__main__' :
    no = 0           # 몇 개의 포스트를 저장했는지 세기 위한 index
    query = "킹크랩 섹션" # 검색을 원하는 문자열로써 UTF-8로 인코딩한다.
    display = 10     # 검색 결과 출력 건수 지정, 10(기본값), 100(최대값)
    start = 1        # 검색 시작 위치로 최대 1000까지 가능
    sort = 'date'    # 정렬 옵션 : sim(유사도순, 기본값), date(날짜순)
    
    #블로그 콘텐츠의 한글 저장을 위해 encoding='utf-8'으로 설정.
    fs = open(query + ".txt",'a',encoding='utf-8')
    
    blog_count = get_blog_count(query,display)
    for start_index in range(start,blog_count + 1, display):
        get_blog_post(query,display,start_index,sort)
        
    fs.close()

Last build date :Sat, 20 Jul 2019 18:20:30 +0900
Total :1121
Start :1
Display :10
Blog total :113
Blog count : 113
-------------------------------------------------
#1
Title :베이비킹크랩 섹션 직접 먹어봤다
Link :https://blog.naver.com/knh514?Redirect=Log&logNo=221393147370
Description :캥크랩은 알지만 베이비킹크랩은 처음 들어봤어요~ 신기신기 직접 먹어본 베이비킹크랩 아이스박스에... 얼음팩과 킹크랩가위 설명서까지 들어있답니다 킹크랩 다리가 도대체 몇마리인지~ 베이비킹크랩 섹션... 
Blogger Name :달콤솜사탕♡
Blogger Link :https://blog.naver.com/knh514
Post Date :20181107
-------------------------------------------------
#2
Title :수산파크 호텔,식당 납품용 냉동대게,킹크랩 섹션 도매 안내
Link :https://blog.naver.com/eastseasusan?Redirect=Log&logNo=221375524424
Description :냉동자숙 대게 섹션은 한박스당 5kg 으로 작업, 킹크랩은 한박스당 7kg 으로 작업했습니다^^ 냉동 자숙 대게섹션, 킹크랩 섹션 도매 납품 문의 주시면 비용, 배송안내드리겠습니다. 출고지는 수입산지인... 
Blogger Name :수산파크
Blogger Link :https://blog.naver.com/eastseasusan
Post Date :20181011
-------------------------------------------------
#3
Title :킹크랩섹션 판매가격,1키로당 45,000원//러시아 캄차카... 
Link :https://blog.naver.com/heesuga?Redirect

-------------------------------------------------
#41
Title :잠실맛집 킹크랩의 계절이 돌아왔다
Link :https://blog.naver.com/sweetstuff33?Redirect=Log&logNo=220523901519
Description :맛난 킹크랩을 먹자는 일념 하나로 또 방문.^^ 홀에는 입탁과 좌탁 섹션 둘다 있어서 편하신 곳으로 앉으심 되구요, 룸도 여러개 있어서 모임하기에도 아주 좋아요 이 집 메뉴는 평일점심땐 점심정식 이런 것도... 
Blogger Name :당신의 비타민 그리고,
Blogger Link :https://blog.naver.com/sweetstuff33
Post Date :20151030
-------------------------------------------------
#42
Title :한정 딱 100박스! [선어(생물) 레드 킹크랩!] 한정 특가 100kg
Link :https://blog.naver.com/sby888?Redirect=Log&logNo=221313046609
Description :[선어(생물) 레드 킹크랩!] 한정 특가 100kg 호텔,뷔페,식당에서도 못가져가서 안달난 역대급... TV홈쇼핑에서 판매하는 몸통 분리된 섹션 대게가 아닌 홀라운드(대게 한마리 전체) 냉동대게입니다.... 
Blogger Name :수산파크 동해수산유통
Blogger Link :https://blog.naver.com/sby888
Post Date :20180705
-------------------------------------------------
#43
Title :실속 상품! 킹크랩 다리만 자숙 판매! 한정판매 딱 3kg
Link :https://blog.naver.com/sby888?Redirect=Log&logNo=221470452988
Description :냉동 섹션 킹크랩이 아닌 생물, 활 킹크랩에서 떨어진 다리를 오늘 바로 자숙(찜)하여 출고하는

-------------------------------------------------
#71
Title :::양양맛집:: 쏠비치 뷔페 엘꼬시네로
Link :https://blog.naver.com/gemma_changg?Redirect=Log&logNo=221480554284
Description :전복이랑 회 킹크랩 차가워서 살짝 아쉽 피자와 카프레제 쌀국수 쌀국수 국물 시원하고 굿굿 초코분수에서 만들어온 거 윤이 아이스크림 케이크와 떡 핫초코 LA갈비랑 메로구이가 진짜 맛있었다 해산물 섹션도... 
Blogger Name :젬마네 집
Blogger Link :https://blog.naver.com/gemma_changg
Post Date :20190305
-------------------------------------------------
#72
Title :인천 가족모임 청라레드크렙에서 생일맞이로 랍스타정식으로... 
Link :https://blog.naver.com/hsjk1004?Redirect=Log&logNo=221537089170
Description :엄청 단독된 룸은 아니지만 섹션이 나누어져서 프라이빗했구요. 다른 테이블에도 나름 섹션이 다... 인천레드크랩 #인천맛집 #인천랍스터 #인천대게 #청라랍스터 #청라대게 #청라킹크랩 #인천킹크랩 #청라맛집... 
Blogger Name :Jackie's paradise ♥
Blogger Link :https://blog.naver.com/hsjk1004
Post Date :20190514
-------------------------------------------------
#73
Title :레드카드바우처 포시즌스호텔뷔페다녀옴
Link :https://blog.naver.com/boma0109?Redirect=Log&logNo=221493295612
Description :입구에는 디저트 섹션과 와인셀러가 있고, 잡지 신문등이 비치되어 있어 식사 후에 따로 카페를 갈... 킹크랩 집게

Link :https://blog.naver.com/thinkerbell2?Redirect=Log&logNo=221434527498
Description :인어같다 꾸물꾸물 해파리 섹션에서는 샤프심으로 점 찍은 것 만한 해파리도 있었고 이렇게 예쁜... 킹크랩 먹고 싶ㅇ... 미안........ 아쿠아리움 마지막 즈음에는 수족관을 따라 무빙사이드워크가 있다.... 
Blogger Name :뇌에서 심장까지
Blogger Link :https://blog.naver.com/thinkerbell2
Post Date :20190104
-------------------------------------------------
#99
Title :쭈꾸미 ,자연산대하 , 참문어통숙회 , 뿔소라 , 킹크랩 , 랍스타  등
Link :https://blog.naver.com/cjdthddl83?Redirect=Log&logNo=220631607294
Description :놓치면ᆞ후회하시는ᆞ박달대게섹션ᆞ특가!!! 끓는물에 5분 정도만 삶고 바로... 2Kg 급 대형 총3마리 기존가격216000원➡ 특가188000원 킹크랩 2.2kg 내외 1마리 ➡138000원 4인분 랍스타 2.2kg... 
Blogger Name :쌍둥이네 010-2984-8256
Blogger Link :https://blog.naver.com/cjdthddl83
Post Date :20160219
-------------------------------------------------
#100
Title :냉동킹크랩 이번주 한시 초특가 방출!!! 쿨러에 냉동킹크랩... 
Link :https://blog.naver.com/tjfao2011?Redirect=Log&logNo=220081952403
Description :-킹크랩 냉동다리,섹션 1키로당 29,000원. *섹션은 다리에 몸통살이 붙어있는 제품입니다. 주문전화 : 1577-6071. 032-561-7003. 문자,카톡 : ***-****-***